In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost.sklearn import XGBRegressor
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.externals import joblib
import numpy
import xlrd

In [17]:
raw_df = pd.read_csv("./dataset/data.csv", header = None)
raw_df.rename(columns = {0:'Ae', 1:'Ap', 2:'Rs', 3:'Fz', 4:'label'}, inplace = True)
pred_df1 = raw_df[0:19]
pred_df2 = raw_df[38:57]

In [18]:
pred_df1

,Ae,Ap,Rs,Fz,label
0,5.0,2.5,4000,0.05,1
1,5.0,2.5,4500,0.05,0
2,5.0,2.5,5000,0.05,0
3,5.0,2.5,5500,0.05,1
4,5.0,2.5,6000,0.05,0
5,5.0,2.5,6500,0.05,1
6,5.0,2.5,7000,0.05,0
7,5.0,2.5,7500,0.05,1
8,5.0,2.5,8000,0.05,0
9,5.0,2.5,8500,0.05,0


In [19]:
pred_df2

,Ae,Ap,Rs,Fz,label
38,5.0,1.5,4000,0.05,4
39,5.0,1.5,4500,0.05,1
40,5.0,1.5,5000,0.05,1
41,5.0,1.5,5500,0.05,3
42,5.0,1.5,6000,0.05,1
43,5.0,1.5,6500,0.05,4
44,5.0,1.5,7000,0.05,1
45,5.0,1.5,7500,0.05,4
46,5.0,1.5,8000,0.05,1
47,5.0,1.5,8500,0.05,1


In [4]:
def re_cate(label):
    if label >2:
        return 1
    else:
        return 0

In [5]:
raw_df['label'] = raw_df.apply(lambda x : re_cate(x.label), axis = 1)

In [6]:
raw_matrix = raw_df.as_matrix()
raw_feature = raw_matrix[:, 0:-2]
raw_label = raw_matrix[:, -1]
raw_feature_mean = raw_feature.mean(axis = 0)
raw_feature_std = raw_feature.std(axis = 0)
raw_feature_normalized = (raw_feature - raw_feature_mean)/raw_feature_std
#np.random.shuffle(raw_feature_normalized)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [20]:
pred_matrix1 = pred_df1.as_matrix()
pred_feature1 = pred_matrix1[:, 0:-2]
pred_feature_normalized1 = (pred_feature1 - raw_feature_mean)/raw_feature_std

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [21]:
pred_matrix2 = pred_df2.as_matrix()
pred_feature2 = pred_matrix2[:, 0:-2]
pred_feature_normalized2 = (pred_feature2 - raw_feature_mean)/raw_feature_std

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [8]:
lr = LogisticRegression(solver='newton-cg', multi_class='multinomial', C=4, tol=1e-6, max_iter=20)

In [9]:
count_CV = 0
test_acc_record = []
test_pre_record = []
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
lr_output_dir = './model_accuracy_jupyter/LR/'

for train_index,test_index in kfold.split(raw_feature_normalized, raw_label):
    lr.fit(raw_feature_normalized[train_index], raw_label[train_index])
    pred_smile_label = lr.predict(raw_feature_normalized[test_index])
    real_label = raw_label[test_index]
  
    TP_num = 0
    FP_num = 0
    FN_num = 0
    TN_num = 0
    
    test_count_num = 0
    real_label_index = 0
    pre_label_num = 0
    
    for label in pred_smile_label:
        if label == real_label[real_label_index]:
            if label == 0:
                TN_num += 1
            if label == 1:
                TP_num += 1
            pre_label_num += 1
        else:
            if label == 0:
                FN_num += 1
            if label == 1:
                FP_num += 1
                
        real_label_index += 1
        test_count_num += 1
    
    print('#### In Cross Validation %d: ####'% count_CV)
    count_CV += 1
    print('NumofIns Precisely Classified : ',pre_label_num,'\t',
          'NumofIns : ',test_count_num,'\t',
          'Pre_Accuracy : ',pre_label_num/test_count_num,'\t',)
    
    test_pre_record.append(pre_label_num/test_count_num)
    model_output_name = lr_output_dir + "model_%d"%(count_CV)
    joblib.dump(lr, model_output_name)

    print("TP:", TP_num)
    print("FP:", FP_num)
    print("FN:", FN_num)
    print("TN:", TN_num)
    accuracy = (TP_num + TN_num)/(TP_num + FP_num + FN_num + TN_num)
    precision = TP_num/(TP_num + FP_num)
    recall = TP_num/(TP_num + FN_num)
    f1 = (2 * precision * recall)/(precision + recall)
    print(accuracy)
    print(precision)
    print(recall)
    print(f1)
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    
print('mean of NumofIns precisely classified',np.mean(test_pre_record))
print('mean of accuracy',np.mean(accuracy_list))
print('mean of precision',np.mean(precision_list))
print('mean of recall',np.mean(recall_list))
print('mean of f1',np.mean(f1_list))

#### In Cross Validation 0: ####
NumofIns Precisely Classified :  27 	 NumofIns :  39 	 Pre_Accuracy :  0.6923076923076923 	
TP: 20
FP: 10
FN: 2
TN: 7
0.6923076923076923
0.6666666666666666
0.9090909090909091
0.7692307692307692
#### In Cross Validation 1: ####
NumofIns Precisely Classified :  24 	 NumofIns :  39 	 Pre_Accuracy :  0.6153846153846154 	
TP: 19
FP: 12
FN: 3
TN: 5
0.6153846153846154
0.6129032258064516
0.8636363636363636
0.7169811320754716
#### In Cross Validation 2: ####
NumofIns Precisely Classified :  27 	 NumofIns :  38 	 Pre_Accuracy :  0.7105263157894737 	
TP: 14
FP: 3
FN: 8
TN: 13
0.7105263157894737
0.8235294117647058
0.6363636363636364
0.717948717948718
#### In Cross Validation 3: ####
NumofIns Precisely Classified :  26 	 NumofIns :  38 	 Pre_Accuracy :  0.6842105263157895 	
TP: 16
FP: 6
FN: 6
TN: 10
0.6842105263157895
0.7272727272727273
0.7272727272727273
0.7272727272727273
#### In Cross Validation 4: ####
NumofIns Precisely Classified :  26 	 NumofIns :  38 	 Pre_A

In [44]:
train_lr_name = './model_accuracy_jupyter/LR/model_1'
trained_lr = joblib.load(train_lr_name)
lr_pred_result1 = trained_lr.predict(pred_feature_normalized1)
print(lr_pred_result1)
lr_pred_result2 = trained_lr.predict(pred_feature_normalized2)
print(lr_pred_result2)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [11]:
SVM = SVC(kernel='rbf',decision_function_shape='ovo',C=20,shrinking =False,tol =1e-6)

In [12]:
count_CV = 0
test_acc_record = []
test_pre_record = []
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
svm_output_dir = './model_accuracy_jupyter/SVM/'

for train_index,test_index in kfold.split(raw_feature_normalized, raw_label):
    SVM.fit(raw_feature_normalized[train_index], raw_label[train_index])
    pred_smile_label = SVM.predict(raw_feature_normalized[test_index])
    real_label = raw_label[test_index]
  
    TP_num = 0
    FP_num = 0
    FN_num = 0
    TN_num = 0
    
    test_count_num = 0
    real_label_index = 0
    pre_label_num = 0
    
    for label in pred_smile_label:
        if label == real_label[real_label_index]:
            if label == 0:
                TN_num += 1
            if label == 1:
                TP_num += 1
            pre_label_num += 1
        else:
            if label == 0:
                FN_num += 1
            if label == 1:
                FP_num += 1
                
        real_label_index += 1
        test_count_num += 1
    
    print('#### In Cross Validation %d: ####'% count_CV)
    count_CV += 1
    print('NumofIns Precisely Classified : ',pre_label_num,'\t',
          'NumofIns : ',test_count_num,'\t',
          'Pre_Accuracy : ',pre_label_num/test_count_num,'\t',)
    
    test_pre_record.append(pre_label_num/test_count_num)
    model_output_name = svm_output_dir + "model_%d"%(count_CV)
    joblib.dump(lr, model_output_name)

    print("TP:", TP_num)
    print("FP:", FP_num)
    print("FN:", FN_num)
    print("TN:", TN_num)
    accuracy = (TP_num + TN_num)/(TP_num + FP_num + FN_num + TN_num)
    precision = TP_num/(TP_num + FP_num)
    recall = TP_num/(TP_num + FN_num)
    f1 = (2 * precision * recall)/(precision + recall)
    print(accuracy)
    print(precision)
    print(recall)
    print(f1)
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    
print('mean of NumofIns precisely classified',np.mean(test_pre_record))
print('mean of accuracy',np.mean(accuracy_list))
print('mean of precision',np.mean(precision_list))
print('mean of recall',np.mean(recall_list))
print('mean of f1',np.mean(f1_list))

#### In Cross Validation 0: ####
NumofIns Precisely Classified :  31 	 NumofIns :  39 	 Pre_Accuracy :  0.7948717948717948 	
TP: 20
FP: 6
FN: 2
TN: 11
0.7948717948717948
0.7692307692307693
0.9090909090909091
0.8333333333333333
#### In Cross Validation 1: ####
NumofIns Precisely Classified :  31 	 NumofIns :  39 	 Pre_Accuracy :  0.7948717948717948 	
TP: 18
FP: 4
FN: 4
TN: 13
0.7948717948717948
0.8181818181818182
0.8181818181818182
0.8181818181818182
#### In Cross Validation 2: ####
NumofIns Precisely Classified :  35 	 NumofIns :  38 	 Pre_Accuracy :  0.9210526315789473 	
TP: 22
FP: 3
FN: 0
TN: 13
0.9210526315789473
0.88
1.0
0.9361702127659575
#### In Cross Validation 3: ####
NumofIns Precisely Classified :  27 	 NumofIns :  38 	 Pre_Accuracy :  0.7105263157894737 	
TP: 18
FP: 7
FN: 4
TN: 9
0.7105263157894737
0.72
0.8181818181818182
0.7659574468085107
#### In Cross Validation 4: ####
NumofIns Precisely Classified :  34 	 NumofIns :  38 	 Pre_Accuracy :  0.8947368421052632 	
TP: 19
FP: 

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

In [42]:
train_svm_name = './model_accuracy_jupyter/SVM/model_1'
trained_svm = joblib.load(train_svm_name)
svm_pred_result1 = trained_svm.predict(pred_feature_normalized1)
print(svm_pred_result1)
svm_pred_result2 = trained_svm.predict(pred_feature_normalized2)
print(svm_pred_result2)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]


In [14]:
XGB_C = XGBClassifier(
    #booster = 'gblinear',
    #objective='multi:softmax',
    #num_class=7,#必须要考虑到0的情况。这个数据集里面没有零
    n_estimators=200,
    max_depth=4,
    min_child_weight = 5,
    scale_pos_weight = 5,
    num_boost_round =5,
    max_delta_step=1000,
    alpha =2,
    eta=1
    #colsample_bytree=0.9
    #gamma=5,
    #process_type='update'
)

In [15]:
count_CV = 0
test_acc_record = []
test_pre_record = []
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
xgboost_output_dir = './model_accuracy_jupyter/XGBoost/'

for train_index,test_index in kfold.split(raw_feature_normalized, raw_label):
    XGB_C.fit(raw_feature_normalized[train_index], raw_label[train_index])
    pred_smile_label = XGB_C.predict(raw_feature_normalized[test_index])
    real_label = raw_label[test_index]
  
    TP_num = 0
    FP_num = 0
    FN_num = 0
    TN_num = 0
    
    test_count_num = 0
    real_label_index = 0
    pre_label_num = 0
    
    for label in pred_smile_label:
        if label == real_label[real_label_index]:
            if label == 0:
                TN_num += 1
            if label == 1:
                TP_num += 1
            pre_label_num += 1
        else:
            if label == 0:
                FN_num += 1
            if label == 1:
                FP_num += 1
                
        real_label_index += 1
        test_count_num += 1
    
    print('#### In Cross Validation %d: ####'% count_CV)
    count_CV += 1
    print('NumofIns Precisely Classified : ',pre_label_num,'\t',
          'NumofIns : ',test_count_num,'\t',
          'Pre_Accuracy : ',pre_label_num/test_count_num,'\t',)
    
    test_pre_record.append(pre_label_num/test_count_num)
    model_output_name = xgboost_output_dir + "model_%d"%(count_CV)
    joblib.dump(lr, model_output_name)
    
    print("TP:", TP_num)
    print("FP:", FP_num)
    print("FN:", FN_num)
    print("TN:", TN_num)
    accuracy = (TP_num + TN_num)/(TP_num + FP_num + FN_num + TN_num)
    precision = TP_num/(TP_num + FP_num)
    recall = TP_num/(TP_num + FN_num)
    f1 = (2 * precision * recall)/(precision + recall)
    print(accuracy)
    print(precision)
    print(recall)
    print(f1)
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    
print('mean of NumofIns precisely classified',np.mean(test_pre_record))
print('mean of accuracy',np.mean(accuracy_list))
print('mean of precision',np.mean(precision_list))
print('mean of recall',np.mean(recall_list))
print('mean of f1',np.mean(f1_list))

#### In Cross Validation 0: ####
NumofIns Precisely Classified :  31 	 NumofIns :  39 	 Pre_Accuracy :  0.7948717948717948 	
TP: 22
FP: 8
FN: 0
TN: 9
0.7948717948717948
0.7333333333333333
1.0
0.846153846153846
#### In Cross Validation 1: ####
NumofIns Precisely Classified :  35 	 NumofIns :  39 	 Pre_Accuracy :  0.8974358974358975 	
TP: 22
FP: 4
FN: 0
TN: 13
0.8974358974358975
0.8461538461538461
1.0
0.9166666666666666
#### In Cross Validation 2: ####
NumofIns Precisely Classified :  31 	 NumofIns :  38 	 Pre_Accuracy :  0.8157894736842105 	
TP: 22
FP: 7
FN: 0
TN: 9
0.8157894736842105
0.7586206896551724
1.0
0.8627450980392156
#### In Cross Validation 3: ####
NumofIns Precisely Classified :  30 	 NumofIns :  38 	 Pre_Accuracy :  0.7894736842105263 	
TP: 22
FP: 8
FN: 0
TN: 8
0.7894736842105263
0.7333333333333333
1.0
0.846153846153846
#### In Cross Validation 4: ####
NumofIns Precisely Classified :  34 	 NumofIns :  38 	 Pre_Accuracy :  0.8947368421052632 	
TP: 21
FP: 3
FN: 1
TN: 13
0.8947

In [43]:
train_xgboost_name = './model_accuracy_jupyter/XGBoost/model_1'
trained_xgboost = joblib.load(train_xgboost_name)
xgboost_pred_result1 = trained_xgboost.predict(pred_feature_normalized1)
print(xgboost_pred_result1)
xgboost_pred_result2 = trained_xgboost.predict(pred_feature_normalized2)
print(xgboost_pred_result2)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]
